# Demo Transformer de HuggingFace para Q&A

https://huggingface.co/models?pipeline_tag=question-answering&language=es&sort=likes

In [1]:
#@title Instalar paquete Transformers de HuggingFace
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.2 MB/s eta 0:00:00


In [2]:
#@title Cargar Modelo Transformer para Q&A 

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch  # nota: usa Torch porque con TF directo tira error

# ver detalles en https://huggingface.co/models?pipeline_tag=question-answering&language=es&sort=likes

nombreModelo = "timpal0l/mdeberta-v3-base-squad2" #@param [ "timpal0l/mdeberta-v3-base-squad2", "mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es", "BSC-LT/roberta-large-bne-sqac"]

# cargar modelo
tokenizer = AutoTokenizer.from_pretrained(nombreModelo)
model = AutoModelForQuestionAnswering.from_pretrained(nombreModelo)


# función para ejecutar el modelo
def responder_pregunta(question, context):
  # prepara input
  inputs = tokenizer(question, context, return_tensors="pt")
  with torch.no_grad():
      outputs = model(**inputs)
  # detrmina rango
  answer_start_index = torch.argmax(outputs.start_logits)
  answer_end_index = torch.argmax(outputs.end_logits)
  # ejecuta el modelo y decodifica
  predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
  return tokenizer.decode(predict_answer_tokens)

import textwrap

In [8]:
#@title Pobar modelo 1

import textwrap
# prueba transformer
question = "de que color es caballo San Martín?" #@param {type:"string"}
context = "San Martín cabalgaba en su caballo blanco cuando ataco al enemigo." #@param {type:"string"}


res = responder_pregunta(question, context)

print("\n")
print("> ", textwrap.fill(context))
print("\t", question)
print("\t  -->", res)




>  ['San Martín cabalgaba en su caballo blanco cuando ataco al enemigo.']
	 de que color es caballo San Martín?
	  --> blanco


In [30]:
#@title Pobar modelo 2

# prueba transformer
question = "qui\xE9n salt\xF3?" #@param {type:"string"}
context = "Luego de saltar sobre sobre la pelota roja, el perro gris cayó dentro de la pileta" #@param {type:"string"}


res = responder_pregunta(question, context)

print("\n")
print("> ", textwrap.fill(context, 100))
print("\t", question)
print("\t  -->", res)




>  Luego de saltar sobre sobre la pelota roja, el perro gris cayó dentro de la pileta
	 quién saltó?
	  --> perro gris
